In [1]:
import tensorflow as tf
assert tf.__version__.startswith('2')
from tensorflow import keras
import numpy as np

2023-11-28 17:06:26.916686: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-28 17:06:27.852199: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import os
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Activation
from keras.optimizers import RMSprop, Adam
from keras.preprocessing import image
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [3]:
data_path = "original_data/"

In [4]:
validation_ratio = 0.1
from keras.preprocessing.image import ImageDataGenerator
train_data_gen = ImageDataGenerator(rescale=1.0/255, validation_split=validation_ratio,zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2, fill_mode='nearest')
train_generator = train_data_gen.flow_from_directory(data_path, target_size=(224,224), batch_size=4, class_mode='categorical', subset='training')
valid_generator = train_data_gen.flow_from_directory(data_path, target_size=(224,224), batch_size=4, class_mode='categorical', subset='validation')

Found 11237 images belonging to 9 classes.
Found 1242 images belonging to 9 classes.


In [5]:
labels = {value: key for key, value in train_generator.class_indices.items()}
for key, value in labels.items():
    print(f"{key} : {value}")

0 : black_spot
1 : black_spot_early
2 : canker
3 : canker_early
4 : greening
5 : greening_early
6 : healthy
7 : melanose
8 : melanose_early


In [6]:
from keras.regularizers import l2

In [7]:
def create_model():
    model = Sequential([
        Conv2D(filters=128, kernel_size=(5, 5), padding='valid', input_shape=(224, 224, 3)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        
        Conv2D(filters=64, kernel_size=(3, 3), padding='valid', kernel_regularizer=l2(0.00005)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        
        Conv2D(filters=32, kernel_size=(3, 3), padding='valid', kernel_regularizer=l2(0.00005)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        
        Flatten(),
        
        Dense(units=256, activation='relu'),
        Dropout(0.5),
        Dense(units=6, activation='softmax')
    ])
    
    return model

In [8]:
cnn_model = create_model()

2023-11-28 17:06:41.226042: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 73 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:03:00.0, compute capability: 7.5


In [9]:
print(cnn_model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 220, 220, 128)     9728      
                                                                 
 activation (Activation)     (None, 220, 220, 128)     0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 110, 110, 128)     0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 110, 110, 128)     512       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 108, 108, 64)      73792     
                                                                 
 activation_1 (Activation)   (None, 108, 108, 64)      0

In [10]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=np.sqrt(0.1), patience=5)

In [11]:
opt1 = Adam(learning_rate=0.001)
cnn_model.compile(optimizer=opt1, loss = "categorical_crossentropy", metrics = ["accuracy"])

In [12]:
hist1 = cnn_model.fit(train_generator, epochs=30, validation_data=valid_generator, verbose=2, callbacks=[reduce_lr])

Epoch 1/30


2023-11-28 17:07:06.542312: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 21.12MiB (rounded to 22151168)requested by op Fill
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-11-28 17:07:06.542364: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-11-28 17:07:06.542386: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 42, Chunks in use: 42. 10.5KiB allocated for chunks. 10.5KiB in use in bin. 4.2KiB client-requested in use in bin.
2023-11-28 17:07:06.542402: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 11, Chunks in use: 11. 5.5KiB allocated for chunks. 5.5KiB in use in bin. 5.5KiB client-requested in use in bin.
2023-11-28 17:07:06.542418: I tensorflow/tsl/framework/bfc_al

ResourceExhaustedError: in user code:

    File "/home/tower1/.local/lib/python3.8/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/home/tower1/.local/lib/python3.8/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/tower1/.local/lib/python3.8/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/home/tower1/.local/lib/python3.8/site-packages/keras/src/engine/training.py", line 1084, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "/home/tower1/.local/lib/python3.8/site-packages/keras/src/optimizers/optimizer.py", line 544, in minimize
        self.apply_gradients(grads_and_vars)
    File "/home/tower1/.local/lib/python3.8/site-packages/keras/src/optimizers/optimizer.py", l